## 2015-2019
Rotterdam Marathon now use the site sporthive.com for the data. Using R-code the data is extracted using the api from the sporthive website

In [1]:
import pandas as pd
import numpy as np

In [48]:
df_2019 = pd.read_csv("data/uitslagen_2019_R.csv", encoding='cp1252')
df_2018 = pd.read_csv("data/uitslagen_2018_R.csv", encoding='cp1252')

In [49]:
#rename agegroprank to agegroupplace
df_2019.rename(columns={"city": "City","Agegrouprank": "AgegroupPlace","bib":"Bib"}, inplace=True)
df_2019.loc[:,'Gender'] = df_2019.loc[:,'Agegroup'].str[0].replace({'V': 'F'})
df_2019.loc[:,'Age'] = df_2019.loc[:,'Agegroup'].str[1:].replace({'SR': '18-34', '35': '35-39', '40': '40-44', '45': '45-49','50':'50-','55':'50-','60':'50-'})

df_2018.rename(columns={"city": "City","Agegrouprank": "AgegroupPlace","bib":"Bib"}, inplace=True)
df_2018['Gender'] = df_2018['Agegroup'].str[0].replace({'V': 'F'})
df_2018['Age'] = df_2018['Agegroup'].str[1:].replace({'SR': '18-34', '35': '35-39', '40': '40-44', '45': '45-49','50':'50-','55':'50-','60':'50-'})

Sporthives uses 2-letter country codes while uitslagen.nl use 3-letter FIFA codes. 
For Sporthyves, using the ISO3166 standard, the 2-letter codes are converted to 3-letter codes.

List is parsed fom https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes

In [50]:
#map 2-letter country names to 3-letter country names
countrycodes = pd.read_csv("data/alpha2-3-countryiso.csv", delimiter=";")

s = countrycodes.set_index('Alpha-2 code')['Alpha-3 code']
df_2019['Country'] = df_2019['countryCode'].map(s)
df_2018['Country'] = df_2018['countryCode'].map(s)

In [51]:
def hms_to_s(s):
    t = 0
    for u in s.split(':'):
        t = 60 * t + int(u)
    return t

In [52]:
distance_list = ("5KM", "10KM", "15KM","20KM", "HALF", "25KM", "30KM", "35KM", "40KM","42KM")

for i in range (1,11):
    x = str(i)
    df_2019['split'+x+'_time_s'] = df_2019['split'+x+'_time'].replace({'nan':np.nan, 0:np.nan, "":np.nan}).fillna("00:00")
    df_2019['split'+x+'_time_s'] = df_2019['split'+x+'_time_s'].astype(str).apply(hms_to_s)
    df_2019['distance_'+distance_list[i-1]+'_time'] = df_2019['split'+x+'_time_s']
    #2018
    df_2018['split'+x+'_time_s'] = df_2018['split'+x+'_time'].replace({'nan':np.nan, 0:np.nan, "":np.nan}).fillna("00:00")
    df_2018['split'+x+'_time_s'] = df_2018['split'+x+'_time_s'].astype(str).apply(hms_to_s)
    df_2018['distance_'+distance_list[i-1]+'_time'] = df_2018['split'+x+'_time_s']

df_2019["Guntime_s"] = df_2019["Guntime"].astype(str).apply(hms_to_s)
df_2019["Chiptime_s"] = df_2019["Chiptime"].astype(str).apply(hms_to_s)
#2018
df_2018["Guntime_s"] = df_2018["Guntime"].astype(str).apply(hms_to_s)
df_2018["Chiptime_s"] = df_2018["Chiptime"].astype(str).apply(hms_to_s)

df_2019.rename(columns={"distance_HALF_time":"HALF"},inplace=True)
df_2018.rename(columns={"distance_HALF_time":"HALF"},inplace=True)

In [53]:
split_list = ("5KM", "10KM", "15KM","20KM","25KM", "30KM", "35KM", "40KM","42KM")
for i in range(0,9):
    if i == 0:
        df_2019['split_'+split_list[i]+'_time'] = df_2019['distance_'+split_list[i]+'_time']
        df_2018['split_'+split_list[i]+'_time'] = df_2018['distance_'+split_list[i]+'_time']
    else:
        df_2019['split_'+split_list[i]+'_time'] = df_2019['distance_'+split_list[i]+'_time'] - df_2019['distance_'+split_list[i-1]+'_time']
        df_2018['split_'+split_list[i]+'_time'] = df_2018['distance_'+split_list[i]+'_time'] - df_2018['distance_'+split_list[i-1]+'_time']

In [54]:
df_2019['Year'] = 2019
df_2018['Year'] = 2018
df_2019['Distance']= "NN Marathon Rotterdam 2019"
df_2018['Distance']= "NN Marathon Rotterdam 2018"
#remove columns not needed or double
df_2019 = df_2019[["Place","Name","City","Country","AgegroupPlace","Agegroup","Guntime_s","Chiptime_s","HALF", 
                         "Bib","Gender","Age","Distance","Year",'split_5KM_time', 'distance_5KM_time', 'split_10KM_time', 'distance_10KM_time', 
                         'split_15KM_time', 'distance_15KM_time', 'split_20KM_time', 'distance_20KM_time', 'split_25KM_time', 
                         'distance_25KM_time', 'split_30KM_time', 'distance_30KM_time', 'split_35KM_time', 'distance_35KM_time',
                         'split_40KM_time', 'distance_40KM_time',"split_42KM_time","distance_42KM_time"]]

df_2018 = df_2018[["Place","Name","City","Country","AgegroupPlace","Agegroup","Guntime_s","Chiptime_s","HALF", 
                         "Bib","Gender","Age","Distance","Year",'split_5KM_time', 'distance_5KM_time', 'split_10KM_time', 'distance_10KM_time', 
                         'split_15KM_time', 'distance_15KM_time', 'split_20KM_time', 'distance_20KM_time', 'split_25KM_time', 
                         'distance_25KM_time', 'split_30KM_time', 'distance_30KM_time', 'split_35KM_time', 'distance_35KM_time',
                         'split_40KM_time', 'distance_40KM_time',"split_42KM_time","distance_42KM_time"]]

df_2019.to_csv("data/uitslagen2019.csv", index = None, header=True, encoding='UTF-8')
df_2018.to_csv("data/uitslagen2018.csv", index = None, header=True, encoding='UTF-8')

In [ ]:
## make the data copied from sporthive.com
# 2019 6509035005995656960/races/450996
# 2018 6386505967023513344/races/419161

import requests
from time import sleep
import csv

req = requests.session()

ongoing = True
url = 'https://eventresults-api.sporthive.com/api/events/6386505967023513344/races/419161/classifications/search?count=50&offset={offset}'
offset = 0
data = []
while ongoing:
    quinquagintuplet = req.get(url.format(offset=offset)).json()
    data.extend(quinquagintuplet['fullClassifications'])
    newbies = len(quinquagintuplet['fullClassifications'])
    if newbies != 50:
        ongoing == False
        break
    offset += newbies
    sleep(0.5)
    print(len(data))

neodata = [{k: d['classification'][k] for k in d['classification'] if k not in ('splits',)} for d in data]

with open('2019.csv','w', newline='') as fh:
    w = csv.DictWriter(fh, fieldnames=list(neodata[1].keys()), extrasaction='ignore')
    w.writeheader()
    w.writerows(neodata)